In [1]:
import pickle

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
%matplotlib inline
import pandas as pd

In [3]:
import os, glob
import seaborn as sns
sns.set(style="darkgrid")

In [4]:
from IPython.display import display, HTML, Image

In [5]:
from sklearn.pipeline import make_pipeline
from yellowbrick.regressor import residuals_plot
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [6]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor


In [7]:
import timeit

In [8]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

F:\anaconda3\envs\chemical\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [10]:
from pandas_profiling import ProfileReport

In [11]:
#data =pd.read_csv('10_PC_02_LHS_500_54854_01_s1_G.csv')
data =pd.read_csv('10_PC_02_LHS_5000_54854_01_s1_G.csv')

In [12]:
data

,T(C),b(CaO),b(SiO2),b(H2O),Vol(aq),pH,nCa(aq),nCa(s),nSi(aq),nSi(s_reac),nPortlandite,nAmor-Sl,mCSHQ,nCa(CSHQ),nSi(CSHQ),nH2O(CSHQ),C/S(CSHQ),nGelPW(CSH)
0,25.0,0.508060,0.304081,3.438876,0.046061,12.47261,0.000934,0.507127,1.424097e-06,0.304080,0.012382,0.0,0.061781,0.494744,0.304080,0.875158,1.627021,0.374278
1,25.0,1.258791,0.566548,4.655774,0.048550,12.47261,0.000984,1.257807,1.501005e-06,0.566546,0.336025,0.0,0.115107,0.921782,0.566546,1.630550,1.627021,0.697336
2,25.0,1.396591,0.697116,4.671613,0.043398,12.47261,0.000880,1.395711,1.341699e-06,0.697115,0.261491,0.0,0.141634,1.134221,0.697115,2.006333,1.627021,0.858048
3,25.0,1.461423,0.327679,7.328584,0.098560,12.47261,0.001998,1.459425,3.047140e-06,0.327676,0.926290,0.0,0.066575,0.533135,0.327676,0.943068,1.627021,0.403321
4,25.0,0.667907,0.389925,7.342961,0.111745,12.47261,0.002265,0.665642,3.454804e-06,0.389922,0.031231,0.0,0.079221,0.634411,0.389922,1.122216,1.627021,0.479938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,25.0,1.173291,0.265235,2.943964,0.025986,12.47261,0.000527,1.172764,8.034066e-07,0.265234,0.741223,0.0,0.053888,0.431541,0.265234,0.763357,1.627021,0.326465
4996,25.0,1.128475,0.411329,7.925439,0.113463,12.47261,0.002300,1.126175,3.507885e-06,0.411325,0.456941,0.0,0.083570,0.669234,0.411325,1.183815,1.627021,0.506282
4997,25.0,1.601654,0.537747,7.424736,0.093018,12.47261,0.001885,1.599769,2.875722e-06,0.537744,0.724849,0.0,0.109255,0.874920,0.537744,1.547654,1.627021,0.661885
4998,25.0,1.655962,0.552659,7.347502,0.090305,12.47261,0.001830,1.654132,2.791928e-06,0.552656,0.754949,0.0,0.112284,0.899183,0.552656,1.590573,1.627021,0.680240


In [13]:
data.columns

Index(['T(C)', ' b(CaO)', ' b(SiO2)', ' b(H2O)', ' Vol(aq)', ' pH', ' nCa(aq)',
       ' nCa(s)', ' nSi(aq)', ' nSi(s_reac)', ' nPortlandite', ' nAmor-Sl',
       ' mCSHQ', ' nCa(CSHQ)', ' nSi(CSHQ)', ' nH2O(CSHQ)', ' C/S(CSHQ)',
       ' nGelPW(CSH)'],
      dtype='object')

In [14]:
data.columns =[col.strip() for col in data.columns]

In [ ]:
profile = ProfileReport(data)
profile

Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#profile.to_file("500DataReport.html")
profile.to_file("5000DataReport.html")

In [ ]:
dataNew = data.copy()
dataNew['ratio'] = dataNew['b(CaO)']/dataNew['b(SiO2)']

In [ ]:
dataNew['ratio'].hist()

In [ ]:
group1 = dataNew[dataNew['ratio']>=1.65]
group2 = dataNew[(dataNew['ratio']<1.65) & (dataNew['ratio']>0.67)]
group3 = dataNew[dataNew['ratio']<=0.67]



In [ ]:
df = data.copy()
fig, ax =plt.subplots(figsize=(10,8))
x1= np.linspace(0.0,0.9,30);
y1 = 0+0.67*x1
x2= np.linspace(0.0,0.9,30);
y2 = 0+1.65*x2
ax.plot(group1['b(SiO2)'].values,group1['b(CaO)'].values,'bo',markerfacecolor = 'tab:olive',markeredgecolor ='darkblue',markeredgewidth=0.5,label='group1')
ax.plot(group2['b(SiO2)'].values,group2['b(CaO)'].values,'bo',markerfacecolor = 'c',markeredgecolor ='b',markeredgewidth=0.5,label='group2')
ax.plot(group3['b(SiO2)'].values,group3['b(CaO)'].values,'g^',markerfacecolor = 'm',markeredgecolor ='m',markeredgewidth=1,label='group3')
ax.plot(x1,y1,'r-',lw=2,label ='Line1:CaO/SiO2 = 0.67')
ax.plot(x2,y2,'y-',lw=3,label ='Line2:CaO/SiO2 = 1.65')
ax.legend (loc='best',ncol=3)           
ax.set_xlim(0,0.9)
ax.set_ylim(0,2)

ax.set_xlabel('b(SiO2')
ax.set_ylabel('b(CaO')



In [ ]:
profile_group1 = ProfileReport(group1)
profile_group1.to_file('Group1_profiling.html')

In [ ]:
profile_group2 = ProfileReport(group2)
profile_group2.to_file('Group2_profiling.html')

In [ ]:
profile_group3 = ProfileReport(group3)
profile_group3.to_file('Group3_profiling.html')